In [10]:
import pulp
import networkx as nx
import plotly.graph_objects as go
%matplotlib notebook

# Model with positive weights

In [11]:
G =nx.Graph()
G.add_edge(1,2,weight=2)
G.add_edge(1,3,weight=2)
G.add_edge(2,3,weight=2)
G.add_edge(2,4,weight=1)
G.add_edge(4,5,weight=1)
G.add_edge(5,3,weight=3)

In [12]:
# Solve as MIP
prob = pulp.LpProblem("max_cut", pulp.LpMaximize)

#################################################################################
# variables

# y[v] = 1 if node v in S, = 0 if node v is in T
y = pulp.LpVariable.dicts(
    "y",
    G.nodes(),
    cat=pulp.LpBinary,
)

# x[(u,v)] = 1 if edge (u,v) in cut
x = pulp.LpVariable.dicts(
    "x",
    G.edges(),
    cat=pulp.LpBinary,
)
"""
# z[(u,v)] = 1 if edge (u,v) in S
z = pulp.LpVariable.dicts(
    "z",
    G.edges(),
    cat=pulp.LpBinary,
)
"""
#################################################################################
# cost function
prob += pulp.lpSum(G[u][v]["weight"]*x[(u,v)] for (u,v) in G.edges())

#################################################################################
# constraints

# node 1 is in S, node 2 is not
#prob += y[1]==1
#prob += y[2]==0

for (u,v) in G.edges():
    # if (u,v) is in the cut, u or v is in S
    prob += x[(u,v)] <= y[u]+y[v]
    # if (u,v) is in the cut, u and v cannot both be in S
    prob += x[(u,v)] + y[u] + y[v] <= 2
    
#################################################################################
# solve
#prob.solve(pulp.CPLEX_CMD(msg=1))
prob.solve()
print(pulp.LpStatus[prob.status])
print("total cost =", pulp.value(prob.objective))

Optimal
total cost = 9.0


In [13]:
pos = nx.spring_layout(G)  # positions for all nodes
nx.draw_networkx(G,pos)
nx.draw_networkx_nodes(G, pos, nodelist=[v for v in G.nodes() if pulp.value(y[v])>0.9], node_color="r")
nx.draw_networkx_edges(
    G,
    pos,
    edgelist=[(u,v) for (u,v) in G.edges() if pulp.value(x[(u,v)])>0.9 ],
    width=8,
    alpha=0.5,
    edge_color="tab:red",
)
nx.draw_networkx_edge_labels(G,pos)

<IPython.core.display.Javascript object>

{(1, 2): Text(0.4695089631186719, -0.45705996802622545, "{'weight': 2}"),
 (1, 3): Text(0.10346093192646667, -0.6066997262825402, "{'weight': 2}"),
 (2, 3): Text(0.12794651532390788, -0.20277705956726386, "{'weight': 2}"),
 (2, 4): Text(0.2608197744429962, 0.47343134934452547, "{'weight': 1}"),
 (3, 5): Text(-0.4833314643036113, -0.04294003197377466, "{'weight': 3}"),
 (4, 5): Text(-0.35045820518452303, 0.6332683769380146, "{'weight': 1}")}

In [14]:
pos

{1: array([ 0.44502338, -0.86098263]),
 2: array([ 0.49399455, -0.0531373 ]),
 3: array([-0.23810152, -0.35241682]),
 4: array([0.027645, 1.      ]),
 5: array([-0.72856141,  0.26653675])}

In [15]:
# https://plotly.com/python/text-and-annotations/

node_color_list = []
for v in G.nodes():
    if pulp.value(y[v])>0.9:
        color="red"
    else:
        color="lightblue"
    node_color_list.append(color)
    
pos = nx.spring_layout(G)

node_x = []
node_y = []
for node in pos:
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    
node_trace = go.Scatter(
    x=node_x, y=node_y,
    hoverinfo='text',
    text=[str(v) for v in G.nodes()],
    mode="markers+text",
    marker=dict(
        #showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        # colorscale='YlGnBu',
        #reversescale=True,
        color= node_color_list,
        size=19,
        colorbar=dict(
            #thickness=15,
            #title='Node Connections',
            #xanchor='left',
            #titleside='right'
        ),
        line_width=2))

edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')



fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                # title= <br>Network graph made with Python',
                #titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="", #"Python code: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", 
                    yref="paper",
                    x=0.005,
                    y=-0.002 
                ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
             )
                )
fig.show()

# Model with positive or negative weights

In [ ]:
G =nx.Graph()
G.add_edge(1,2,weight=2)
G.add_edge(1,3,weight=-2)
G.add_edge(2,3,weight=2)
G.add_edge(2,4,weight=1)
G.add_edge(4,5,weight=-1)
G.add_edge(5,3,weight=3)

In [ ]:
# Solve as MIP
prob = pulp.LpProblem("max_cut", pulp.LpMaximize)

#################################################################################
# variables

# y[v] = 1 if node v in S, = 0 if node v is in T
y = pulp.LpVariable.dicts(
    "y",
    G.nodes(),
    cat=pulp.LpBinary,
)

# x[(u,v)] = 1 if edge (u,v) in cut
x = pulp.LpVariable.dicts(
    "x",
    G.edges(),
    cat=pulp.LpBinary,
)

# z[(u,v)] = 1 if edge (u,v) in S
z = pulp.LpVariable.dicts(
    "z",
    G.edges(),
    cat=pulp.LpBinary,
)

#################################################################################
# cost function
prob += pulp.lpSum(G[u][v]["weight"]*x[(u,v)] for (u,v) in G.edges())

#################################################################################
# constraints

# node 1 is in S, node 2 is not
#prob += y[1]==1
#prob += y[2]==0

for (u,v) in G.edges():
    # if (u,v) is in the cut, u or v is in S
    prob += x[(u,v)] +2*z[(u,v)] == y[u]+y[v]
    
#################################################################################
# solve
#prob.solve(pulp.CPLEX_CMD(msg=1))
prob.solve()
print(pulp.LpStatus[prob.status])
print("total cost =", pulp.value(prob.objective))

Optimal
total cost = 7.0


In [ ]:
pos = nx.spring_layout(G)  # positions for all nodes
nx.draw_networkx(G,pos)
nx.draw_networkx_nodes(G, pos, nodelist=[v for v in G.nodes() if pulp.value(y[v])>0.9], node_color="r")
nx.draw_networkx_edges(
    G,
    pos,
    edgelist=[(u,v) for (u,v) in G.edges() if pulp.value(x[(u,v)])>0.9 ],
    width=8,
    alpha=0.5,
    edge_color="tab:red",
)
nx.draw_networkx_edge_labels(G,pos)

{(1, 2): Text(-0.4197016607590382, 0.09560152524292848, "{'weight': 2}"),
 (1, 3): Text(0.08280120580040462, 0.03030366577852718, "{'weight': -2}"),
 (2, 3): Text(0.45516586924557717, -0.029183495504825425, "{'weight': 2}"),
 (2, 4): Text(-0.5236684986569328, 0.08009260115920404, "{'weight': 1}"),
 (3, 5): Text(0.9197016607590383, -0.15763694442234458, "{'weight': 3}"),
 (4, 5): Text(-0.05913270714347163, -0.04836084775831511, "{'weight': -1}")}

In [3]:
pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 52.2 MB/s eta 0:00:00


In [4]:
pip install networkx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import networkx as nx
import pulp

In [1]:
# Min Cut

In [27]:
G =nx.DiGraph()
G.add_edge('a','b',weight=2)
G.add_edge('a','c',weight=3)
G.add_edge('c','b',weight=3)
G.add_edge('c','d',weight=2)
G.add_edge('b','d',weight=5)

In [33]:
# Solve as MIP
prob = pulp.LpProblem("min_cut", pulp.LpMinimize)

#################################################################################
# variables

# y[v] = 1 if node v in S, = 0 if node v is in T
y = pulp.LpVariable.dicts(
    "y",
    G.nodes(),
    cat=pulp.LpBinary,
)

# x[(u,v)] = 1 if edge (u,v) in cut
x = pulp.LpVariable.dicts(
    "x",
    G.edges(),
    cat=pulp.LpBinary,
)

#################################################################################
# cost function
prob += pulp.lpSum(G[u][v]["weight"]*x[(u,v)] for (u,v) in G.edges())

#################################################################################
# constraints

# node 1 is in S, node 2 is not
prob += y['a']==1
prob += y['d']==0

#

for (u,v) in G.edges():
    prob +=  y[u] <= y[v]+x[(u,v)]



    
#################################################################################
# solve
#prob.solve(pulp.CPLEX_CMD(msg=1))
prob.solve()
print(pulp.LpStatus[prob.status])
print("total cost =", pulp.value(prob.objective))

Optimal
total cost = 5.0


In [34]:
for (u,v) in x:
  if pulp.value(x[(u,v)])> 0.9:
    print(u,v)

a b
a c


In [35]:
for u in y:
  if pulp.value(y[u])>0.9:
    print(u)

a


In [36]:
prob

min_cut:
MINIMIZE
2*x_('a',_'b') + 3*x_('a',_'c') + 5*x_('b',_'d') + 3*x_('c',_'b') + 2*x_('c',_'d') + 0
SUBJECT TO
_C1: y_a = 1

_C2: y_d = 0

_C3: - x_('a',_'b') + y_a - y_b <= 0

_C4: - x_('a',_'c') + y_a - y_c <= 0

_C5: - x_('b',_'d') + y_b - y_d <= 0

_C6: - x_('c',_'b') - y_b + y_c <= 0

_C7: - x_('c',_'d') + y_c - y_d <= 0

VARIABLES
0 <= x_('a',_'b') <= 1 Integer
0 <= x_('a',_'c') <= 1 Integer
0 <= x_('b',_'d') <= 1 Integer
0 <= x_('c',_'b') <= 1 Integer
0 <= x_('c',_'d') <= 1 Integer
0 <= y_a <= 1 Integer
0 <= y_b <= 1 Integer
0 <= y_c <= 1 Integer
0 <= y_d <= 1 Integer